In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

In [ ]:
print(tf.version.VERSION)

# Load Model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('./path-classifier.h5', compile=False)

# Determine Input and Output Sizes

In [ ]:
model.inputs

In [ ]:
model.outputs

# Convert to TensorFlow Model and Freeze the Tensorflow Session 

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ''
        frozen_graph = convert_variables_to_constants(session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [ ]:
frozen_graph = freeze_session(K.get_session(), output_names=[out.op.name for out in model.outputs])

# Save TensorFlow Model to be Deployed

In [ ]:
tf.train.write_graph(frozen_graph, ".", "path-classifier.pb", as_text=False)